## 环境配置

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import optuna
import optuna.visualization as ov
from joblib import dump, load
import numpy as np

## 数据准备

In [2]:
# 数据提取
df_train = pd.read_csv("./datasets/train.csv")
df_test = pd.read_csv("./datasets/test.csv")

# 重命名
df_train.columns = (
        ['index', 'datetime'] +
        [f'V{i + 1}' for i in range(17)] + [f'T1-{i + 1}' for i in range(17)] + [f'T2-{i + 1}' for i in range(17)] +
        [f'T1R-{i + 1}' for i in range(17)] + [f'T2R-{i + 1}' for i in range(17)]

)
df_test.columns = df_train.columns[:53]
df_train

,index,datetime,V1,V2,V3,V4,V5,V6,V7,V8,...,T2R-8,T2R-9,T2R-10,T2R-11,T2R-12,T2R-13,T2R-14,T2R-15,T2R-16,T2R-17
0,1,2022/11/6 9:08,35.668999,36.146000,25.558001,26.195000,25.670000,15.702,16.690001,15.991,...,827,827,827,827,827,827,827,827,827,750
1,2,2022/11/6 9:09,35.995998,36.347000,25.382000,26.348000,26.131001,15.523,16.825001,15.871,...,827,827,827,827,827,827,827,827,827,750
2,3,2022/11/6 9:11,35.340000,36.311001,25.469999,26.093000,25.639000,15.564,15.564000,15.947,...,827,827,827,827,827,827,827,827,827,750
3,4,2022/11/6 9:12,35.585999,36.091000,25.250000,26.127001,25.670000,15.575,16.775999,15.936,...,827,827,827,827,827,827,827,827,827,750
4,5,2022/11/6 9:13,35.946999,36.256001,25.163000,26.399000,25.837999,15.460,16.580999,15.795,...,827,827,827,827,827,827,827,827,827,750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26651,26652,2023/3/1 3:49,24.594000,24.377001,29.191999,25.551001,27.016001,4.377,21.929001,24.459,...,837,837,837,837,837,837,837,837,837,750
26652,26653,2023/3/1 3:54,24.379000,24.424999,29.253000,25.652000,27.188000,4.415,22.017000,24.534,...,837,837,837,837,837,837,837,837,837,750
26653,26654,2023/3/1 4:00,24.407000,24.312000,29.010000,25.382000,26.813000,4.354,21.726000,24.204,...,837,837,837,837,837,837,837,837,837,750
26654,26655,2023/3/1 4:05,24.636000,24.409000,29.162001,25.551001,27.032000,4.362,21.813000,21.813,...,837,837,837,837,837,837,837,837,837,750


## 特征工程

In [3]:
def generate_features(df, feature_types: [str, ...]):
    df['timestamp'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('timestamp')

    # 时间特征
    if 'time' in feature_types:
        df['year'] = df['timestamp'].dt.year
        df['month'] = df['timestamp'].dt.month
        df['day'] = df['timestamp'].dt.day
        df['hour'] = df['timestamp'].dt.hour
        df['minute'] = df['timestamp'].dt.minute
        df['dayofweek'] = df['timestamp'].dt.dayofweek

    # 累积统计特征
    if 'statistical' in feature_types:
        for i in range(1, 18):
            df[f'T1-{i}_mean'] = df[[f'T1-{j}' for j in range(1, i + 1)]].mean(axis=1)
            df[f'T1-{i}_std'] = df[[f'T1-{j}' for j in range(1, i + 1)]].std(axis=1)

            df[f'T2-{i}_mean'] = df[[f'T2-{j}' for j in range(1, i + 1)]].mean(axis=1)
            df[f'T2-{i}_std'] = df[[f'T2-{j}' for j in range(1, i + 1)]].std(axis=1)

            df[f'V{i}_mean'] = df[[f'V{j}' for j in range(1, i + 1)]].mean(axis=1)
            df[f'V{i}_std'] = df[[f'V{j}' for j in range(1, i + 1)]].std(axis=1)

    # 上下温度区总体特征
    if 'overall' in feature_types:
        # Average, min, max, and standard deviation
        df['T1_mean'] = df[[f'T1-{i}' for i in range(1, 18)]].mean(axis=1)
        df['T1_min'] = df[[f'T1-{i}' for i in range(1, 18)]].min(axis=1)
        df['T1_max'] = df[[f'T1-{i}' for i in range(1, 18)]].max(axis=1)
        df['T1_std'] = df[[f'T1-{i}' for i in range(1, 18)]].std(axis=1)

        df['T2_mean'] = df[[f'T2-{i}' for i in range(1, 18)]].mean(axis=1)
        df['T2_min'] = df[[f'T2-{i}' for i in range(1, 18)]].min(axis=1)
        df['T2_max'] = df[[f'T2-{i}' for i in range(1, 18)]].max(axis=1)
        df['T2_std'] = df[[f'T2-{i}' for i in range(1, 18)]].std(axis=1)

        # Difference and ratio between upper and lower heating rods
        df['T1_T2_diff'] = df['T1_mean'] - df['T2_mean']
        df['T1_T2_ratio'] = df['T1_mean'] / df['T2_mean']

    # 上下温度区交互特征
    if 'interaction' in feature_types:
        for i in range(1, 18):
            df[f'T1-{i}_T2-{i}_diff'] = df[f'T1-{i}'] - df[f'T2-{i}']
            df[f'T1-{i}_T2-{i}_ratio'] = df[f'T1-{i}'] / df[f'T2-{i}']
            df[f'T1-{i}_T2-{i}_interaction'] = df[f'T1-{i}'] * df[f'T2-{i}']

    # 相邻数据点的差值
    if 'difference' in feature_types:
        for i in range(1, 17):
            df[f'T1-{i}_T1-{i + 1}_diff'] = df[f'T1-{i}'] - df[f'T1-{i + 1}']
            df[f'T2-{i}_T2-{i + 1}_diff'] = df[f'T2-{i}'] - df[f'T2-{i + 1}']
            df[f'V{i}_V{i + 1}_diff'] = df[f'V{i}'] - df[f'V{i + 1}']

    # 比例特征
    if 'ratio' in feature_types:
        for i in range(1, 18):
            df[f'T1-{i}_V{i}_ratio'] = df[f'T1-{i}'] / df[f'V{i}']
            df[f'T2-{i}_V{i}_ratio'] = df[f'T2-{i}'] / df[f'V{i}']
            df[f'T1-{i}_V{i}_interaction'] = df[f'T1-{i}'] * df[f'V{i}']
            df[f'T2-{i}_V{i}_interaction'] = df[f'T2-{i}'] * df[f'V{i}']

    # 滑动窗口特征
    if 'rolling_window' in feature_types:
        window_sizes = [i for i in range(1, 5)]  # Change this list according to your needs
        for window_size in window_sizes:
            for i in range(1, 18):
                df[f'T1-{i}_rolling_mean_{window_size}'] = df[f'T1-{i}'].rolling(window_size).mean()
                df[f'T2-{i}_rolling_mean_{window_size}'] = df[f'T2-{i}'].rolling(window_size).mean()
                df[f'V{i}_rolling_mean_{window_size}'] = df[f'V{i}'].rolling(window_size).mean()

    # 空间梯度特征
    # 假设加热棒之间的空间距离是均匀的
    if 'spatial_gradient' in feature_types:
        for i in range(1, 17):
            df[f'T1-{i}_T1-{i + 1}_gradient'] = (df[f'T1-{i + 1}'] - df[f'T1-{i}']) / i
            df[f'T2-{i}_T2-{i + 1}_gradient'] = (df[f'T2-{i + 1}'] - df[f'T2-{i}']) / i

    # 时间梯度特征
    if 'time_gradient' in feature_types:
        for i in range(1, 18):
            df[f'T1-{i}_time_gradient'] = df[f'T1-{i}'].diff()
            df[f'T2-{i}_time_gradient'] = df[f'T2-{i}'].diff()
            df[f'V{i}_time_gradient'] = df[f'V{i}'].diff()

    # 时空梯度特征
    if 'time_spatial_gradient' in feature_types:
        for i in range(1, 17):
            df[f'T1-{i}_T1-{i + 1}_time_gradient'] = df[f'T1-{i + 1}'].diff() - df[f'T1-{i}'].diff()
            df[f'T2-{i}_T2-{i + 1}_time_gradient'] = df[f'T2-{i + 1}'].diff() - df[f'T2-{i}'].diff()
            df[f'V{i}_V{i + 1}_time_gradient'] = df[f'V{i + 1}'].diff() - df[f'V{i}'].diff()

    return df


# 要使用的特征类型
#
# use_feature_types = ['time', 'statistical', 'overall', 'interaction', 'difference', 'ratio',
#                      'rolling_window',
#                      'spatial_gradient', 'time_gradient', 'time_spatial_gradient']
use_feature_types = ['time']
df_add = generate_features(df_train.copy(), use_feature_types)
df_add_test = generate_features(df_train.copy(), use_feature_types)

df_add.columns

Index(['index', 'datetime', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8',
       'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'T1-1',
       'T1-2', 'T1-3', 'T1-4', 'T1-5', 'T1-6', 'T1-7', 'T1-8', 'T1-9', 'T1-10',
       'T1-11', 'T1-12', 'T1-13', 'T1-14', 'T1-15', 'T1-16', 'T1-17', 'T2-1',
       'T2-2', 'T2-3', 'T2-4', 'T2-5', 'T2-6', 'T2-7', 'T2-8', 'T2-9', 'T2-10',
       'T2-11', 'T2-12', 'T2-13', 'T2-14', 'T2-15', 'T2-16', 'T2-17', 'T1R-1',
       'T1R-2', 'T1R-3', 'T1R-4', 'T1R-5', 'T1R-6', 'T1R-7', 'T1R-8', 'T1R-9',
       'T1R-10', 'T1R-11', 'T1R-12', 'T1R-13', 'T1R-14', 'T1R-15', 'T1R-16',
       'T1R-17', 'T2R-1', 'T2R-2', 'T2R-3', 'T2R-4', 'T2R-5', 'T2R-6', 'T2R-7',
       'T2R-8', 'T2R-9', 'T2R-10', 'T2R-11', 'T2R-12', 'T2R-13', 'T2R-14',
       'T2R-15', 'T2R-16', 'T2R-17', 'timestamp', 'year', 'month', 'day',
       'hour', 'minute', 'dayofweek'],
      dtype='object')

## 训练前预处理

In [4]:
target_cols = df_add.columns[53: 53 + 17 * 2].tolist()
feature_cols = [col for col in df_add.columns if col not in target_cols + ['index', 'datetime', 'timestamp']]
X = df_add[feature_cols]
X_test = df_add_test[feature_cols]
y = df_add[target_cols]

# 压缩内存
y

,T1R-1,T1R-2,T1R-3,T1R-4,T1R-5,T1R-6,T1R-7,T1R-8,T1R-9,T1R-10,...,T2R-8,T2R-9,T2R-10,T2R-11,T2R-12,T2R-13,T2R-14,T2R-15,T2R-16,T2R-17
0,244,312,399,500,567,647,730,827,827,827,...,827,827,827,827,827,827,827,827,827,750
1,244,312,399,500,567,647,731,827,827,827,...,827,827,827,827,827,827,827,827,827,750
2,245,312,399,500,567,647,731,827,827,827,...,827,827,827,827,827,827,827,827,827,750
3,245,312,399,500,567,647,732,827,827,827,...,827,827,827,827,827,827,827,827,827,750
4,245,312,399,500,567,647,732,827,827,827,...,827,827,827,827,827,827,827,827,827,750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26651,306,374,447,521,576,659,735,837,837,837,...,837,837,837,837,837,837,837,837,837,750
26652,306,375,447,520,576,658,736,837,837,837,...,837,837,837,837,837,837,837,837,837,750
26653,306,375,448,521,576,658,735,837,837,837,...,837,837,837,837,837,837,837,837,837,750
26654,306,374,448,521,576,659,735,837,837,837,...,837,837,837,837,837,837,837,837,837,750


## 模型训练

In [ ]:
NUM_ROUND = 3

### 超参数搜索

In [ ]:
def objective(trial, x, y_target):
    # 超参数范围
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'mae',
            'min_child_weight': 5,
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'lambda_l2': 10,
            'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'seed': 2023,
            'nthread': 16,
            'verbose': -1,
    }

    # 5-Fold 交叉验证
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    mae_scores_target = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = x.iloc[train_index], x.iloc[val_index]
        y_train, y_val = y_target.iloc[train_index], y_target.iloc[val_index]

        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

        gbm = lgb.train(params, train_data, valid_sets=val_data, num_boost_round=NUM_ROUND)

        y_val_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
        mae = mean_absolute_error(y_val, y_val_pred)

        mae_scores_target.append(mae)

    # 平均mae
    average_mae = np.mean(mae_scores_target)
    trial.report(average_mae, step=0)

    #
    return average_mae

### 多目标训练

In [ ]:

def optimize_by_multi_targets():
    for i, target in enumerate(y.columns):
        y_target = y[target]

        # 设定目标函数
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, X, y_target), n_trials=100)

        # Get the best hyperparameters
        best_params = study.best_trial.params
        best_params['boosting_type'] = 'gbdt'
        best_params['objective'] = 'regression_l1'
        best_params['metric'] = 'mae'
        best_params['verbose'] = 0

        # Train and save a model using the best hyperparameters
        train_data = lgb.Dataset(X, label=y_target)
        gbm_best = lgb.train(best_params, train_data, num_boost_round=5000)

        dump(gbm_best, f'./models/model_{i}.joblib')

        # 打印
        print('Best trial:')
        best_trial = study.best_trial

        print(f'Value: {best_trial.value}')

        print('Params:')
        for key, value in best_trial.params.items():
            print(f'{key}: {value}')

        # 可视化
        ov.plot_optimization_history(study).show()
        ov.plot_param_importances(study).show()


optimize_by_multi_targets()

## 模型推理

In [ ]:
def predict_by_multi_targets():
    # Initialize an empty dataframe to store the predictions
    df_predictions = pd.DataFrame()

    # For each target, load the corresponding model and make predictions
    for i in range(len(y.columns)):
        # Load the model
        gbm_best_saved = load(f'model_{i}.joblib')

        # Make predictions
        predictions = gbm_best_saved.predict(X_test)

        # Store the predictions in the dataframe
        df_predictions[f'target_{i}'] = predictions

    # Save the predictions to a csv file
    df_predictions.to_csv('predictions.csv', index=False)

